# Playground

# TensorFlow Lite Converter

## Import Dependencies

In [23]:
# general 
import numpy as np

# for model re-building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


## Setup

In [24]:
# Define sequence length
SEQ_LENGTH = 30
# Define total number of keypoints
N_KEYPOINTS = 1662

# Define list of actions used
actions = np.array(['hello', 'thanks', 'iloveyou'])


## Load Trained Model

In [25]:
# re-initialize the model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(SEQ_LENGTH, N_KEYPOINTS))) # input_shape=(#sequences, #keypoints)
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.load_weights('tutorial/action.h5')

In [26]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [27]:
# Save the entire model as a SavedModel.
%mkdir -p saved_model
model.save('saved_model/action_model')

INFO:tensorflow:Assets written to: saved_model/action_model/assets


INFO:tensorflow:Assets written to: saved_model/action_model/assets


In [28]:
model = tf.keras.models.load_model('saved_model/action_model')

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

## Convert to TensorFlow Lite

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()

open("saved_model/action_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/tg/gv0w34yj4575fd8wlc9nv1k40000gn/T/tmp7htszxmq/assets


INFO:tensorflow:Assets written to: /var/folders/tg/gv0w34yj4575fd8wlc9nv1k40000gn/T/tmp7htszxmq/assets
2023-04-18 23:56:31.222866: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-18 23:56:31.222879: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-04-18 23:56:31.467272: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:2051] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexTensorListReserve, FlexTensorListSetItem, FlexTensorListStack
Details:
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x128xf32>>>) : {device = ""}
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x64xf32>>>) : {device = ""}
	tf.TensorListSetItem(tensor<!tf_type.variant<tensor<?x128xf32>>>, tensor<i32>, tensor<?x128xf32>) -> (tensor<!tf_typ

2413748